# Hugging Face Causal Language Model

fine-tuning of the [Facebook/opt-125](https://huggingface.co/facebook/opt-125m) model using an portuguese dataset [mc4-pt-sample-1g.txt](https://unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt) of 300 million tokens in its causal language modeling pre-training. The opt-125 model was originally trained on a english dataset of approximately 300 bil\lion tokens.

The task is to predict the next token in a sequence, like GPT and not like BERT

[![google colab link](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tcvieira/IA368-DD-012023/blob/main/assingments/04/notebook.ipynb)

# Installs

In [29]:
!pip install transformers -q
!pip install datasets -q
!pip install ipython-autotime -q
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 12.5 s (started: 2023-03-30 02:44:26 +00:00)


In [30]:
!pip install numba -q

time: 4.2 s (started: 2023-03-30 02:44:39 +00:00)


# Imports

In [31]:
from datasets import load_dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
    AutoConfig
)
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

time: 3.38 ms (started: 2023-03-30 02:44:43 +00:00)


# Dataset

In [32]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive
time: 5.14 s (started: 2023-03-30 02:44:43 +00:00)


In [33]:
PATH_DATASET = '/content/drive/MyDrive/unicamp/IA368DD/class_4'

time: 410 µs (started: 2023-03-30 02:44:48 +00:00)


In [34]:
#!gsutil cp gs://unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt {PATH_DATASET}/sample-1gb.txt

time: 241 µs (started: 2023-03-30 02:44:48 +00:00)


In [35]:
!head {PATH_DATASET}/sample-1gb.txt

Linkbar Há alguns anos, o número de rapazes e moças que subiam ao púlpito para pregar era maior que o de hoje. Na sua simplicidade, falavam do amor de Deus, da Salvação e davam testemunho sob a unção do Espirito Santo. Hoje, parece que a figura do "preletor oficial" inibiu muitos de falarem com ousadia a Palavra de Deus. Parece que há um receio de falar diante de um público que, certamente, é mais intelectualizado que há alguns anos. Jovens pregadores ficam embaraçados e cometem certos deslizes, que poderiam ser evitados. Neste modesto trabalho, vamos dar apenas algumas sugestões, e não um estudo sobre a Homilética (Arte de Falar em Publico). I -O QUE PREGAR? É a comunicação verbal da Palavra de Deus aos ouvintes. É a transmissão do evangelho de Nosso Senhor Jesus Cristo às pessoas que precisam ouvi-lo. II- QUAL A FINALIDADE DA PREGAÇÃO? É persuadir as pessoas a aceitarem a mensagem da Palavra de Deus para sua salvação (descrentes) ou para seu crescimento espiritual (crentes). Diante d

In [36]:
!wc -l {PATH_DATASET}/sample-1gb.txt

250000 /content/drive/MyDrive/unicamp/IA368DD/class_4/sample-1gb.txt
time: 784 ms (started: 2023-03-30 02:44:48 +00:00)


## small dataset for testing

In [37]:
#!sed -n '1,100p' {PATH_DATASET}/sample-1gb.txt > {PATH_DATASET}/sample_small.txt

time: 252 µs (started: 2023-03-30 02:44:49 +00:00)


In [38]:
!wc -l {PATH_DATASET}/sample_small.txt

100 /content/drive/MyDrive/unicamp/IA368DD/class_4/sample_small.txt
time: 182 ms (started: 2023-03-30 02:44:49 +00:00)


In [39]:
small_dataset = load_dataset("text", data_files=f'{PATH_DATASET}/sample_small.txt')
small_dataset["validation"] = load_dataset("text", data_files=f'{PATH_DATASET}/sample_small.txt', split=f"train[:5%]")
small_dataset["train"] = load_dataset("text", data_files=f'{PATH_DATASET}/sample_small.txt', split=f"train[5%:]")

small_dataset

  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 95
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 5
    })
})

time: 875 ms (started: 2023-03-30 02:44:49 +00:00)


In [40]:
base_dataset = load_dataset("text", data_files=f'{PATH_DATASET}/sample-1gb.txt')
base_dataset["validation"] = load_dataset("text", data_files=f'{PATH_DATASET}/sample-1gb.txt', split=f"train[:20%]")
base_dataset["train"] = load_dataset("text", data_files=f'{PATH_DATASET}/sample-1gb.txt', split=f"train[20%:]")

base_dataset

  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 50000
    })
})

time: 884 ms (started: 2023-03-30 02:44:50 +00:00)


# Select Dataset

In [41]:
#dataset = small_dataset
dataset = base_dataset

time: 341 µs (started: 2023-03-30 02:44:51 +00:00)


# Parameters

In [42]:
MODEL_NAME = 'facebook/opt-125m'
MAX_SEQ_LENGTH=512
BATCH_SIZE=8
EPOCHS=2
MODEL_OUTPUT_FOLDER=f'{PATH_DATASET}/model_output'
MODEL_SAVE_FOLDER=f'{PATH_DATASET}/model_save'
TOKENIZER_SAVE_FOLDER=f'{PATH_DATASET}/tokenizer_save'

time: 519 µs (started: 2023-03-30 02:44:51 +00:00)


# Tokenizer

In [43]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

time: 415 ms (started: 2023-03-30 02:44:51 +00:00)


# Model

`AutoModelForCausalLM` is used for auto-regressive language models like all the GPT models, while `AutoModelForSeq2SeqLM` is used for language models with encoder-decoder architecture like T5 and BART.

https://huggingface.co/docs/transformers/model_doc/auto#transformers.AutoModelForCausalLM

In [44]:
# Download configuration from huggingface.co and cache.
#config = AutoConfig.from_pretrained(MODEL_NAME)
#model = AutoModelForCausalLM.from_config(config)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
model.to(device)

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 768, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 768)
      (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0): OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05,

time: 2.07 s (started: 2023-03-30 02:44:52 +00:00)


In [45]:
model_size = sum(t.numel() for t in model.parameters())
print(f"OPT-125m size: {model_size/1000**2:.1f}M parameters")

OPT-125m size: 125.2M parameters
time: 1.55 ms (started: 2023-03-30 02:44:54 +00:00)


# Tokenization

The model was pretrained with max sequence length of 2048. We use length 256 initially to overcome the exceeded ram problem.

CHANGE: we tokenize all samples in the batch (consisting of 1000 documents) and create a long sequence of tokens by concatenating all examples and separating them with the special EOS token. Finally, we divide the long sequence into chunks of 512 tokens, which will be used for training.

In [46]:
tokenized_dataset = dataset.map(lambda x: tokenizer(x["text"], 
                                      truncation=True, 
                                      padding="max_length", 
                                      max_length=MAX_SEQ_LENGTH), 
                                      batched=True, 
                                      num_proc=4, 
                                      remove_columns=["text"],
                                      #return_overflowing_tokens=True,
                                      #return_length=True,
                                     )

time: 96.9 ms (started: 2023-03-30 02:44:54 +00:00)


In [47]:
#tokenized_dataset.save_to_disk(TOKENIZER_SAVE_FOLDER)

Saving the dataset (0/2 shards):   0%|          | 0/200000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/50000 [00:00<?, ? examples/s]

time: 3.05 s (started: 2023-03-30 02:44:54 +00:00)


In [22]:
#tokenized_dataset = load_dataset(TOKENIZER_SAVE_FOLDER)

time: 227 µs (started: 2023-03-30 02:41:52 +00:00)


In [20]:
# print(tokenized_dataset)
# print(f"{len(tokenized_dataset['train']['input_ids'][0])} tokens - {tokenized_dataset['train']['input_ids'][0]}")

time: 228 µs (started: 2023-03-30 02:27:06 +00:00)


# Training

In [48]:
# The training is done using an T4 with 16GB of memory
!nvidia-smi

Thu Mar 30 02:44:57 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    55W / 400W |   2507MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [29]:
from numba import cuda 

device = cuda.get_current_device()
device.reset()

time: 762 ms (started: 2023-03-30 02:35:44 +00:00)


In [49]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir=MODEL_OUTPUT_FOLDER,
                                  num_train_epochs=EPOCHS, 
                                  per_device_train_batch_size=BATCH_SIZE,
                                  per_device_eval_batch_size=BATCH_SIZE, 
                                  evaluation_strategy="epoch", 
                                  save_strategy="epoch",
                                  logging_strategy="epoch", 
                                  learning_rate=2e-5, 
                                  weight_decay=0.01,
                                  fp16=True # Use mixed precision
                                )

time: 2.8 ms (started: 2023-03-30 02:44:57 +00:00)


In [50]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
trainer = Trainer(model=model, 
                  args=training_args, 
                  train_dataset=tokenized_dataset["train"],
                  eval_dataset=tokenized_dataset["validation"], 
                  data_collator=data_collator)

time: 5.62 ms (started: 2023-03-30 02:44:57 +00:00)


In [51]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,2.571000,2.339543
2,2.327500,2.269327


Exception ignored in: <generator object Json._generate_tables at 0x7f5ed3aeb200>
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/datasets/packaged_modules/json/json.py", line 158, in _generate_tables
    batch_idx += 1
OSError: [Errno 107] Transport endpoint is not connected


TrainOutput(global_step=50000, training_loss=2.449279375, metrics={'train_runtime': 5575.0933, 'train_samples_per_second': 71.748, 'train_steps_per_second': 8.968, 'total_flos': 1.045168128e+17, 'train_loss': 2.449279375, 'epoch': 2.0})

time: 1h 32min 55s (started: 2023-03-30 02:44:57 +00:00)


In [52]:
model.save_pretrained(MODEL_SAVE_FOLDER)
tokenizer.save_pretrained(MODEL_SAVE_FOLDER)

('/content/drive/MyDrive/unicamp/IA368DD/class_4/model_save/tokenizer_config.json',
 '/content/drive/MyDrive/unicamp/IA368DD/class_4/model_save/special_tokens_map.json',
 '/content/drive/MyDrive/unicamp/IA368DD/class_4/model_save/vocab.json',
 '/content/drive/MyDrive/unicamp/IA368DD/class_4/model_save/merges.txt',
 '/content/drive/MyDrive/unicamp/IA368DD/class_4/model_save/added_tokens.json',
 '/content/drive/MyDrive/unicamp/IA368DD/class_4/model_save/tokenizer.json')

time: 1.6 s (started: 2023-03-30 04:19:47 +00:00)


In [58]:
#!zip model {MODEL_SAVE_FOLDER}/*

time: 283 µs (started: 2023-03-29 22:55:03 +00:00)


# Evaluation

In [53]:
model = AutoModelForCausalLM.from_pretrained(MODEL_SAVE_FOLDER)
trainer = Trainer(
    model = model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    data_collator = data_collator
)

time: 1.99 s (started: 2023-03-30 04:19:52 +00:00)


In [54]:
# eval_results = trainer.evaluate()
# perplexity = torch.exp(torch.tensor(eval_results["eval_loss"]))
# print(f'Validation Perplexity =  {perplexity.item():.2f}')

Validation Perplexity =  9.67
time: 2min 40s (started: 2023-03-30 04:20:00 +00:00)


In [55]:
import math

print(f"Validation Perplexity: {math.exp(2.269327):.3f}")

Validation Perplexity: 9.673
time: 911 µs (started: 2023-03-30 04:22:46 +00:00)


In [28]:
# eval_results = trainer.evaluate(tokenized_dataset["train"])
# perplexity = torch.exp(torch.tensor(eval_results["eval_loss"]))
# print(f'Train Perplexity =  {perplexity.item():.2f}')

time: 272 µs (started: 2023-03-30 02:24:17 +00:00)


In [56]:
prompt = 'Era uma vez na fazenda '
inputs = tokenizer(prompt, return_tensors='pt')

# Generate
generate_ids = model.generate(inputs.input_ids.to(device), max_length=30)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'Era uma vez na fazenda irmã de um homem que se encontrava com um pai. Ele'

time: 1.48 s (started: 2023-03-30 04:26:04 +00:00)


In [58]:
from transformers import pipeline

pipe = pipeline('text-generation', model=MODEL_SAVE_FOLDER, tokenizer=tokenizer, max_length=30)

time: 1.96 s (started: 2023-03-30 04:27:07 +00:00)


In [59]:
#print(pipe("Era uma vez na fazenda ", num_return_sequences=1)[0]["generated_text"])

Era uma vez na fazenda irmã de um homem que se encontrava com um pai. Ele est
time: 375 ms (started: 2023-03-30 04:27:19 +00:00)


In [60]:
#print(pipe("era uma vez na fazenda", num_return_sequences=1, max_length=200)[0]["generated_text"])

prompt = "Era uma vez na fazenda "
for _ in range(10):
    print(pipe(prompt, max_length=50, do_sample=True)[0]["generated_text"])

Era uma vez na fazenda irmã, e seu vizinho e o vizinho que comanda o salão de restaurantes com a cachoeira e a borboleta. Na frent
Era uma vez na fazenda irmão o destacando e os dias de hoje fomei meus olhos de manho que a chuva deu conta de que meu am
Era uma vez na fazenda irmão de Paulo que, no início dos anos 90, tanto na família do empresário, como na família dos comerci
Era uma vez na fazenda és outra até o nome do Sr. Manoel, com quem era um jovem és a primeira a conhecer minha cidade.
Era uma vez na fazenda irmão que desafiu a vida para poder se dedicar ao povo que ele tinha. Ele, por sua vez, foi um grande
Era uma vez na fazenda ilesa do sossego Zey, um pediatra que fez a reativação dos dentes fazendo parte da rotina com as crian
Era uma vez na fazenda Â nãos que, hoje em dia, se escondeu à beira de uma linda família. Não é assim. Um grupo
Era uma vez na fazenda irmã do casal que pessoas de coração amada e muito amadora deu àquelas que gostavam de se amare
Era uma vez na fazen

In [62]:
prompt = "Era uma vez na fazenda "
for _ in range(10):
    print(pipe(prompt, max_length=50, do_sample=True, num_beams=5, early_stopping=True, no_repeat_ngram_size=2)[0]["generated_text"])


Era uma vez na fazenda irmã, que a gente ficou muito feliz com o trabalho de nossa família. Estávamos juntos em u
Era uma vez na fazenda Ângela, na cidade de Santo Antônio de Jesus, no Rio de Janeiro, que se encontrava com a família de um homem
Era uma vez na fazenda irmã de um grupo de amigos que estavam na cidade de Florianópolis, nos Estados Unidos. Eles se encontrav
Era uma vez na fazenda irmã de um grupo de pessoas que trabalhavam como lanchonete. Eles tinham um grande público, que
Era uma vez na fazenda Ângela, no bairro São João, que eu tinha oportunidade de conhecer um pouco mais sobre a histó
Era uma vez na fazenda irmã de um jovem de 19 anos que acabou de morar em um apartamento na cidade de São José dos Campos, no interior de
Era uma vez na fazenda irmã de um jovem de 16 anos que, após a morte de sua mãe, decidiu seguir para a cidade de
Era uma vez na fazenda irmã de um jovem de 20 anos, que se encontrava em um apartamento na Zona Norte de São Paulo. A vítima
Era uma vez na 

In [61]:
# Leonardo Augusto da Silva Pacheco - https://github.com/leonardo3108/IA368dd/blob/main/exercicios/Aula_5/Aula_5_Treino_Modelo_de_Linguagem.ipynb

prompt = 'Era uma vez na fazenda '
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenizer(text=prompt)['input_ids']
    input_ids_truncated = input_ids[-MAX_SEQ_LENGTH:]  # Usamos apenas os últimos tokens como entrada para o modelo.
    output = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = output['logits'][:, -1, :]  # Usamos apenas o ultimo token da sequencia
    predicted_id = torch.argmax(logits).item()  # extraindo o token de maior probabilidade (greedy decoding)
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt.replace('</s>', ''))

Era uma vez na fazenda irm
Era uma vez na fazenda irmã
Era uma vez na fazenda irmã de
Era uma vez na fazenda irmã de um
Era uma vez na fazenda irmã de um am
Era uma vez na fazenda irmã de um amigo
Era uma vez na fazenda irmã de um amigo de
Era uma vez na fazenda irmã de um amigo de um
Era uma vez na fazenda irmã de um amigo de um dos
Era uma vez na fazenda irmã de um amigo de um dos m
Era uma vez na fazenda irmã de um amigo de um dos mais
Era uma vez na fazenda irmã de um amigo de um dos mais important
Era uma vez na fazenda irmã de um amigo de um dos mais importantes
Era uma vez na fazenda irmã de um amigo de um dos mais importantes adv
Era uma vez na fazenda irmã de um amigo de um dos mais importantes advog
Era uma vez na fazenda irmã de um amigo de um dos mais importantes advogados
Era uma vez na fazenda irmã de um amigo de um dos mais importantes advogados do
Era uma vez na fazenda irmã de um amigo de um dos mais importantes advogados do Bras
Era uma vez na fazenda irmã de um amigo

# Results

split train/validation (200.000, 50.000)

| seq length | epochs | batch size |    gpu    | time/epoch | val_ppl |
|:----------:|:------:|:----------:|:---------:|:----------:|:-------:|
|     256    |    1   |      4     |  T4 16GB  |     ~2h    |  31.67  |
|     512    |    1   |      4     |  T4 16GB  |     OOM    |    -    |
|     512    |    1   |      8     | A100 40GB |    45min   |  10.96  |
|     512    |   1+1  |      8     | A100 40GB | colab fail |    -    |
|     512    |    2   |      8     | A100 40GB |     ~2h    |   9.67  |
|    1024    |    1   |     16     | A100 40GB |     OOM    |    -    |

----